In [1]:
import os
import sys
from importnb import Notebook
import re
import pandas as pd
import numpy as np
import zipfile
from enum import Enum
import zlib
from dataclasses import dataclass, make_dataclass, field
from typing import Optional, Union, List, Callable
from bs4 import BeautifulSoup

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)
    
with Notebook():
    from src.parserbase import *

In [2]:
class Sav(GamCreBase):
    
    def __init__(self, savefile:str, game:str='BGEE'):
        self.savefile = os.path.join(PKG_ROOT, 'saves', 'original', savefile)
        resourceDir = os.path.join(PKG_ROOT, 'resources', game)
        self.zipfile = zipfile.ZipFile(self.savefile)
        self.filelist = self.zipfile.namelist()
        self.files = [self.zipfile.read(_) for _ in self.filelist]
        self.gamStr = self.files[1].decode('latin')
        self.gamBuffer = bytearray(self.files[1])
        self.gamVersion = re.findall(r'GAME\s*V\d+\.\d+', self.gamStr)[0]  #always 'GAMEV2.0'
        self.creVersion = re.findall(r'CRE\s*V\d+\.\d+', self.gamStr)[0]  #always 'CRE V1.0'
        self.GAM = GAM(resourceDir=resourceDir, buffer=self.gamBuffer)
        
        
        

In [12]:
s = Sav('TESTSAVE.bg1save', 'BGEE')

AttributeError: 'GAM' object has no attribute 'bufferSize'

In [2]:
gamseg_df = pd.read_csv(os.path.join(PKG_ROOT,'resources', 'bgee', 'gamsegs.csv'), index_col=0)
creseg_df = pd.read_csv(os.path.join(PKG_ROOT,'resources', 'bgee', 'cresegs.csv'), index_col=0)
cretable_df = pd.read_csv(os.path.join(PKG_ROOT,'resources', 'bgee', 'cretables.csv'), index_col=0)
res_df = pd.read_csv(os.path.join(PKG_ROOT,'resources', 'formats', 'refkeywords.csv'), index_col=0)

In [10]:
gamseg_df

,SubSegs,CountLoc,OffsetLoc,SizeLoc,SizeValue,CountValue
0,Header,NaN,NaN,NaN,180.0,NaN
1,Party,0x0024,0x0020,NaN,NaN,NaN
2,Inventory,0x002c,0x0028,NaN,NaN,NaN
3,NPC,0x0034,0x0030,NaN,NaN,NaN
4,Global,0x003c,0x0038,NaN,NaN,NaN
5,Journal,0x004c,0x0050,NaN,NaN,NaN
6,FamiliarInfo,NaN,0x0068,NaN,400.0,NaN
7,StoredLocations,0x0070,0x006c,NaN,NaN,NaN
8,PocketPlane,0x007c,0x0078,NaN,NaN,NaN
9,FamiliarExtra,NaN,0x0048,NaN,648.0,NaN


In [11]:
s.GAM.SEGS

NameError: name 's' is not defined